In [ ]:
import os
import ast
import json
import numpy as np
import pandas as pd
from sentence_transformers import SentenceTransformer

# —— Parameters —— #
INPUT_CSV    = 'input/cachedLaws_1.csv'    # your source CSV
QUERY_FILE   = 'input/query.txt'           # your query text
OUTPUT_CSV   = 'output/filtered_laws.csv'  # where to save results
THRESHOLD    = 0.5                         # similarity cutoff (0–1)

# —— Prepare output directory —— #
os.makedirs(os.path.dirname(OUTPUT_CSV), exist_ok=True)

# —— Load data —— #
# Read first 5 rows (tab-separated)
df = pd.read_csv(INPUT_CSV).head(5)
df.to_csv("input/sample_input.csv", index=False)

# Load the user query
with open(QUERY_FILE, 'r', encoding='utf-8') as f:
    query = f.read().strip()

# —— Initialize embedder —— #
model = SentenceTransformer('jinaai/jina-embeddings-v2-small-en')
query_emb = model.encode(query, convert_to_numpy=True)

# —— Processing function —— #
def filter_structured(sj):
    """
    Parse the Python-style JSON in sj, compute cosine similarity of each
    article/annex text to the query, add 'score' and keep only elements
    with score >= THRESHOLD.
    """
    # Parse Python-dict string
    try:
        data = ast.literal_eval(sj)
    except (ValueError, SyntaxError):
        return {'articles': [], 'annexes': []}

    out = {'articles': [], 'annexes': []}
    for section in ('articles', 'annexes'):
        for elem in data.get(section, []):
            text = elem.get('text', '')
            # embed and compute cosine similarity
            emb = model.encode(text, convert_to_numpy=True)
            score = float(
                np.dot(query_emb, emb) /
                (np.linalg.norm(query_emb) * np.linalg.norm(emb))
            )
            elem['score'] = score
            if score >= THRESHOLD:
                out[section].append(elem)
    return out

# —— Apply filtering —— #
df['filtered_json'] = df['structured_json'].apply(filter_structured)

# Keep only rows with at least one match
mask = df['filtered_json'].apply(lambda d: len(d['articles']) + len(d['annexes']) > 0)
df_filtered = df[mask].copy()

# Convert filtered dict back to JSON string
df_filtered['filtered_json'] = df_filtered['filtered_json'].apply(json.dumps)

# Select relevant columns and save
result = df_filtered[['celex_id', 'reference', 'summary', 'filtered_json']]
result.to_csv(OUTPUT_CSV, index=False)
